In [ ]:
# ============================================================================= 流程 [B] ============================================================================
# 1. 指定 selenium 連線到特定網址
# 2. 用 selenium 解析 HTML 結構，取得標籤
# 3. 針對取得的標籤進行操作
# ============================================================================= 流程 [E] ============================================================================

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
import time
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
import pymysql

# 連結 SQL
connect_db = pymysql.connect(host="", port=, user='', passwd='', charset='utf8', db='')

# ============================================================================ driver設定 [B] ============================================================================

options = Options()
options.add_argument("--disable-notifications")
driver = webdriver.Chrome('./chromedriver', chrome_options=options)



# ============================================================================ driver設定 [E] ============================================================================




# ===========================================================================爬蟲程式開始===========================================================================


# 進到網頁，目標：Google學術研究系統
# 教授
# driver.get("https://scholar.google.com/citations?user=CP46e2MAAAAJ&hl=en")
# 教授
driver.get("https://scholar.google.com/citations?hl=en&user=XFm-7ncAAAAJ")

time.sleep(1)
# -----------------------------------------------------------------------------------------------------------
# 先將所有的"更多內容按鈕"按完
# -----------------------------------------------------------------------------------------------------------
while True:
    
    try:
        
        btn = driver.find_element(By.XPATH, '/html/body/div[1]/div[12]/div[2]/div/div[4]/form/div[2]/div/button/span/span[2]')
#         利用按鈕顏色的變化來判斷還能不能按
        if btn.value_of_css_property("color")=='rgba(184, 184, 184, 1)':
#         讓用戶知道按鈕不能再按了，然後離開迴圈
            print('按鈕已經被禁用')
            break
        else:
            btn = driver.find_element(By.XPATH, '/html/body/div[1]/div[12]/div[2]/div/div[4]/form/div[2]/div/button/span/span[2]')
            btn.click()
            time.sleep(1.5)  
    except:
        break
# -------------------------------------------------------------------------------------------------------------------------------------    
#     開始print網頁的內容
# -------------------------------------------------------------------------------------------------------------------------------------            
try: 
    name = driver.find_elements(By.ID, 'gsc_prf_in') #找到所有 class="gsc_prf_in" 的標籤，會是一個 list
    
    title = driver.find_elements(By.CLASS_NAME, 'gsc_a_at') #找到所有 class="gsc_a_at" 的標籤，會是一個 list
    print(title[0].text) #     利用len()找到論文的數量，以便後面for迴圈使用
    len_title = len(title)
    print(len_title)#     在這邊先輸出一下看看
    year = driver.find_elements(By.CSS_SELECTOR, 'span.gsc_a_h.gsc_a_hc.gs_ibl') #利用selector找到所有標籤，會是一個 list
    quote = driver.find_elements(By.XPATH, '/html/body/div[1]/div[12]/div[2]/div/div[4]/form/div[1]/table/tbody/tr[1]/td[2]/a')
    print("教授名稱:", name[0].text)
#     這邊就是利用之前取得的論文數量來放到range裡面
    for i in range(0, 5):
        
        print("論文", i+1, "名稱:", title[i].text, "年份:", year[i].text)
# -------------------------------------------------------------------------------------------------------------------------------------    
#     第三階段
# -------------------------------------------------------------------------------------------------------------------------------------    

#         點進去論文的網頁
#         利用上面存取的論文的名稱來找要點擊的那個按鈕
        btn_title = driver.find_element(By.LINK_TEXT, title[i].text)
        btn_title.click()
        time.sleep(1.5)
        
#         爬摘要的內容
#         這邊一定要try,except因為有些教授的論文是沒有放摘要(description)的
        try:
#         用find_element不是find_elements
#         因為我們這個頁面內只有這麼一個摘要要爬下來，不需要用到list 
            description = driver.find_element(By.CLASS_NAME, 'gsh_small')
            coauthor = driver.find_element(By.XPATH, '/html/body/div[1]/div[7]/div[2]/div/div[2]/div[1]/div[2]')
            journal = driver.find_element(By.XPATH, '/html/body/div[1]/div[7]/div[2]/div/div[2]/div[3]/div[2]')
            
            print("摘要:",description.text)
            print(" ")
            
            with connect_db.cursor() as cursor:
#                 sql = """
#                 INSERT INTO data1 (gid, scholar_name, title) VALUES ( %s, %s, %s); 
#                 """
#                 , abstract, co_author, journal, quote, publish_year, last_crawler_time
# , description.text, coauthor.text, journal.text, quote[i].text, year[i].text, time.ctime()
                
                val = (title[i].text )
                sql = (f"""INSERT INTO data1 ( title) VALUES ('{val}') ;""")
                cursor.execute(sql)
                connect_db.commit()
                
        except Exception as e:
            print("出現了未預期的錯誤：",e)    
            pass
#         回到上一頁
        driver.back()
        time.sleep(2)
        
        
    
        
#     connect_db.close()
except Exception as e:
    print("出現了未預期的錯誤：",e)     

# ===========================================================================爬蟲程式結束========================================================